# SGAI models (DQN)

This notebook is based off of the pytorch tutorial [here](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html). It is intended to both create and train models for Courtney2-Outbreak

### Setup

In [1]:
import sys
import numpy as np
import tensorflow as tf
import keras.layers as layers
import keras.models as models
import keras
from collections import namedtuple
from queue import deque
import random
import math
from tqdm import tqdm  # used for progress meters
sys.path.append("./")  # make sure that it is able to import Board

from Board import Board
from constants import *
from Player import ZombiePlayer, GovernmentPlayer

In [2]:
DEVICE = "GPU"
#tf.debugging.set_log_device_placement(True)
devices = tf.config.list_physical_devices(DEVICE)
print(devices)
tf.config.experimental.set_memory_growth(devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Training Environments

In [3]:
class ZombieEnvironment:
    ACTION_SPACE = tuple(range(8))
    ACTION_MAPPINGS = {
        0: "moveUp",
        1: "moveDown",
        2: "moveLeft",
        3: "moveRight",
        4: "biteUp",
        5: "biteDown",
        6: "biteLeft",
        7: "biteRight",
    }
    SIZE = (6, 6)

    def __init__(self, max_timesteps: int = 300, logdir: str = "", run_name="") -> None:
        self.max_timesteps = max_timesteps
        self.reset()
        self.total_timesteps = 0
        self.total_invalid_moves = 0
        self.writer = None
        if logdir != "" and run_name != "":
            self.writer = tf.summary.create_file_writer(f"{logdir}/{run_name}")

    def reset(self):
        self.board = Board(ZombieEnvironment.SIZE, "Zombie")
        self.board.populate(num_zombies=1)
        self.enemyPlayer = GovernmentPlayer()
        self.done = False

        # coordinates of the first zombie
        self.agentPosition = self.board.indexOf(True)

        # useful for metrics
        self.max_number_of_zombies = 1
        self.episode_invalid_actions = 0
        self.episode_reward = 0
        self.episode_timesteps = 0

        return self._get_obs()

    def step(self, action: int):
        action_name = ZombieEnvironment.ACTION_MAPPINGS[action]
        if "move" in action_name:
            valid, new_pos = self.board.actionToFunction[action_name](
                self.board.toCoord(self.agentPosition)
            )
            if valid:
                self.agentPosition = new_pos
        else:  # bite variation
            dest_coord = list(self.board.toCoord(self.agentPosition))
            if action_name == "biteUp":
                dest_coord[1] -= 1
            elif action_name == "biteDown":
                dest_coord[1] += 1
            elif action_name == "biteRight":
                dest_coord[0] += 1
            else:
                dest_coord[0] -= 1
            valid, _ = self.board.actionToFunction["bite"](dest_coord)

        won = None
        # do the opposing player's action if the action was valid.
        if valid:
            _action, coord = self.enemyPlayer.get_move(self.board)
            if not _action:
                self.done = True
                won = True
            else:
                self.board.actionToFunction[_action](coord)
            self.board.update()

        # see if the game is over
        if not self.board.States[
            self.agentPosition
        ].person.isZombie:  # zombie was cured
            self.done = True
            won = False
        if not self.board.is_move_possible_at(self.agentPosition):  # no move possible
            self.done = True
        if self.episode_timesteps > self.max_timesteps:
            self.done = True

        # get obs, reward, done, info
        obs, reward, done, info = (
            self._get_obs(),
            self._get_reward(action_name, valid, won),
            self._get_done(),
            self._get_info(),
        )

        # update the metrics
        self.episode_reward += reward
        if not valid:
            self.episode_invalid_actions += 1
            self.total_invalid_moves += 1
        self.episode_timesteps += 1
        self.max_number_of_zombies = max(
            self.board.num_zombies(), self.max_number_of_zombies
        )
        self.total_timesteps += 1

        # write the metrics
        if self.writer is not None:
            with self.writer.as_default():
                tf.summary.scalar(
                    "train/invalid_action_rate",
                    self.total_invalid_moves / self.total_timesteps,
                    step=self.total_timesteps,
                )
                tf.summary.scalar("train/cur_reward", reward, step=self.total_timesteps)

        # return the obs, reward, done, info
        return obs, reward, done, info

    def _get_info(self):
        return {}

    def _get_done(self):
        return self.done

    def _get_reward(self, action_name: str, was_valid: bool, won: bool):
        if not was_valid:
            return -100
        if won is True:
            return 100
        if won is False:
            return -100
        if "bite" in action_name:
            return 15
        return 1  # this is the case where it was move

    def _get_obs(self):
        ret = self.board.get_board()
        ret[self.agentPosition] = 4
        return np.array(ret)

    def render(self):
        import PygameFunctions as PF
        import pygame

        PF.run(self.board)
        pygame.display.update()

    def init_render(self):
        import PygameFunctions as PF
        import pygame

        PF.initScreen(self.board)
        pygame.display.update()

    def close(self):
        import pygame

        pygame.quit()

    def write_run_metrics(self):
        if self.writer is not None:
            with self.writer.as_default():
                tf.summary.scalar(
                    "episode/num_invalid_actions_per_ep",
                    self.episode_invalid_actions,
                    step=self.total_timesteps,
                )
                tf.summary.scalar(
                    "episode/episode_length", self.episode_timesteps, step=self.total_timesteps
                )
                tf.summary.scalar(
                    "episode/episode_total_reward",
                    self.episode_reward,
                    step=self.total_timesteps,
                )
                tf.summary.scalar(
                    "episode/mean_reward",
                    self.episode_reward / self.episode_timesteps,
                    step=self.total_timesteps,
                )
                tf.summary.scalar(
                    "episode/percent_invalid_per_ep",
                    self.episode_invalid_actions / self.episode_timesteps,
                    step=self.total_timesteps,
                )


### Make models

In [4]:
ZOMBIE_OUTPUT_SIZE = len(ZombieEnvironment.ACTION_SPACE)
INPUT_SHAPE = (ROWS * COLUMNS,)


In [5]:
def make_zombie_model():
    """
    makes the model that will be used for zombies
    The output of the model will be the predicted q value
    for being in a certain state.
    """
    model = models.Sequential()
    model.add(layers.InputLayer(INPUT_SHAPE))
    model.add(layers.Flatten())
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(64))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(ZOMBIE_OUTPUT_SIZE))
    return model


In [6]:
with tf.device(DEVICE):
    zombie_policy = make_zombie_model()
    zombie_target = make_zombie_model()


In [7]:
print(zombie_policy.input_shape)
zombie_policy.summary()


(None, 36)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 36)                0         
                                                                 
 sync_batch_normalization (S  (None, 36)               144       
 yncBatchNormalization)                                          
                                                                 
 dense (Dense)               (None, 64)                2368      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128)               0         
                                             

In [8]:
# make sure the output is correct shape
with tf.device(DEVICE):
    temp = zombie_policy(tf.random.normal((1, 36)), training=False)
print(temp.shape)


(1, 8)


### Load saved model

In [9]:
#zombie_policy.load_weights("zombie_policy_weights")
#zombie_target.load_weights("zombie_policy_weights")

### DQN utilities

In [10]:
# this acts as a class; useful in the training
Transition = namedtuple("Transition", ("state", "action", "next_state", "reward"))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


### Optimizers and Loss

In [11]:
with tf.device(DEVICE):
    optimizer = keras.optimizers.Adam(0.004)
    loss = keras.losses.MeanSquaredError()


### Training loop

In [12]:
BATCH_SIZE = 128
GAMMA = 0.999
EPSILON = 0.05  # exploration rate
TARGET_UPDATE = 5  # how many episodes before the target is updated

BUFFER_CAPACITY = 10000
memory = ReplayMemory(BUFFER_CAPACITY)

In [13]:
def select_zombie_action(state):
    sample = random.random()
    if sample > EPSILON:
        # Pick the action with the largest expected reward.
        temp = zombie_policy(state, training=False)
        numpy = temp.numpy().flatten()
        return tf.constant([tuple(numpy).index(max(numpy))])
    else:
        return tf.constant([random.randrange(ZOMBIE_OUTPUT_SIZE)], dtype=tf.int32)


In [48]:
from typing import List

def train_on_batch(state_batch:tf.Tensor, action_batch:tf.Tensor, reward_batch:tf.Tensor, non_final_next_states:tf.Tensor, non_final_mask:tf.Tensor, ):
    with tf.GradientTape() as policy_tape:
        # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
        # columns of actions taken. These are the actions which would've been taken
        # for each batch state according to policy_net
        action_batch = tf.expand_dims(action_batch, 1)
        state_action_values = tf.gather_nd(zombie_policy(state_batch, training=True), action_batch, 1)

        # Compute V(s_{t+1}) for all next states.
        # Expected values of actions for non_final_next_states are computed based
        # on the "older" target_net; selecting their best reward with max(1)[0].
        # This is merged based on the mask, such that we'll have either the expected
        # state value or 0 in case the state was final.
        next_state_values = tf.scatter_nd(
            tf.expand_dims(non_final_mask, 1),
            tf.reduce_max(zombie_target(non_final_next_states, training=False), 1),
            tf.constant([BATCH_SIZE]),
        )

        # Compute the expected Q values
        expected_state_action_values = tf.squeeze((next_state_values * GAMMA) + reward_batch)

        # compute loss (mean squared error)
        assert state_action_values.shape == expected_state_action_values.shape
        _loss = loss(state_action_values, expected_state_action_values)

    # Optimize the model
    policy_gradient = policy_tape.gradient(_loss, zombie_policy.trainable_variables)
    optimizer.apply_gradients(zip(policy_gradient, zombie_policy.trainable_variables))


In [49]:
def train(epochs, max_timesteps=200, logdir="", run_name=""):
    env = ZombieEnvironment(max_timesteps, logdir, run_name)
    env.init_render()
    for episode in tqdm(range(epochs)):
        # Initialize the environment and state
        prev_obs = env.reset()
        done = False
        timesteps = 0
        while not done:
            env.render()
            # Select and perform an action
            action = select_zombie_action(tf.constant([prev_obs]))
            action = action.numpy()[0]  # "flatten" the tensor and take the item
            new_obs, reward, done, _ = env.step(action)
            # reward = tf.constant([reward])

            # Observe new state
            if not done:
                next_state = new_obs
            else:
                next_state = None

            # Store the transition in memory
            memory.push(prev_obs, action, next_state, reward)

            # Move to the next state
            prev_obs = next_state

            # Perform one step of the optimization (on the policy network)
            if len(memory) >= BATCH_SIZE:
                # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
                # detailed explanation). This converts batch-array of Transitions
                # to Transition of batch-arrays.
                batch = Transition(*zip(*memory.sample(BATCH_SIZE)))

                # compute the states that aren't terminal states
                non_final_mask = tf.constant(tuple(idx for state, idx in zip(batch.next_state, range(len(batch.next_state))) if state is not None),)
                non_final_next_states = tf.cast(tuple(state for state in batch.next_state if state is not None), dtype=tf.float32)

                train_on_batch(
                    tf.concat([batch.state], 0),
                    tf.concat([batch.action], 0),
                    tf.cast([batch.reward], dtype=tf.float32),
                    non_final_next_states,
                    non_final_mask
                )

        env.write_run_metrics()

        # Update the target network, copying all weights and biases in DQN
        if episode % TARGET_UPDATE == 0:
            zombie_policy.save_weights("zombie_policy_weights")
            # zombie_target.load_state_dict(zombie_policy.state_dict())
            zombie_target.load_weights("./zombie_policy_weights")
    env.close()


### Start Training!

In [50]:
with tf.device(DEVICE):
    train(50, 300, "zombieEnvironment", "run1")


  0%|          | 0/50 [00:00<?, ?it/s]

action is (128,)
action is (128, 1)
before gathering, state is (128, 8)
after gathering, state is (128,)
expected is (1, 128)
expected is now (128,)
action is (128,)
action is (128, 1)
before gathering, state is (128, 8)
after gathering, state is (128,)
expected is (1, 128)
expected is now (128,)
action is (128,)
action is (128, 1)
before gathering, state is (128, 8)
after gathering, state is (128,)
expected is (1, 128)
expected is now (128,)
action is (128,)
action is (128, 1)
before gathering, state is (128, 8)
after gathering, state is (128,)
expected is (1, 128)
expected is now (128,)
action is (128,)
action is (128, 1)
before gathering, state is (128, 8)
after gathering, state is (128,)
expected is (1, 128)
expected is now (128,)
action is (128,)
action is (128, 1)
before gathering, state is (128, 8)
after gathering, state is (128,)
expected is (1, 128)
expected is now (128,)
action is (128,)
action is (128, 1)
before gathering, state is (128, 8)
after gathering, state is (128,)
e

  0%|          | 0/50 [00:01<?, ?it/s]

action is (128,)
action is (128, 1)
before gathering, state is (128, 8)
after gathering, state is (128,)
expected is (1, 128)
expected is now (128,)


KeyboardInterrupt: 